# Visualizing Day 8 as Feature Dashboard 

In [0]:
SELECT * FROM ecom.silver.events

In [0]:
CREATE OR REPLACE TABLE ecom.gold.popular_products AS
SELECT category, brand, COUNT(DISTINCT product_id) AS product_count
FROM ecom.bronze.events
WHERE brand IS NOT NULL AND category IS NOT NULL
GROUP BY category, brand
ORDER BY product_count DESC
LIMIT 20;

CREATE OR REPLACE TABLE ecom.gold.pair_products AS
SELECT
  p1.product_id AS product_id_1,
  p2.product_id AS product_id_2
FROM (
  SELECT DISTINCT session_id, product_id
  FROM ecom.bronze.events
  WHERE event_type = 'purchased'
) p1
JOIN (
  SELECT DISTINCT session_id, product_id
  FROM ecom.bronze.events
  WHERE event_type = 'purchased'
) p2
ON p1.session_id = p2.session_id
WHERE p1.product_id < p2.product_id

In [0]:
CREATE OR REPLACE TABLE ecom.silver.Dedup_events AS
SELECT *
FROM (
    SELECT *, row_number() OVER (PARTITION BY user_id, session_id, event_time, event_type ORDER BY product_id) AS rn
    FROM ecom.bronze.events
    WHERE event_type IN ('view', 'cart', 'purchase')
      AND user_id IS NOT NULL
      AND session_id IS NOT NULL
      AND product_id IS NOT NULL
) sub
WHERE rn = 1;

In [0]:
describe table ecom.silver.Dedup_events

In [0]:
CREATE OR REPLACE TABLE ecom.gold.week_sales_ratio AS
SELECT 
    event_type,
    SUM(price) AS total_price,
    CASE WHEN dayofweek(event_date) IN (1,7)
      THEN 'weekend'
      ELSE 'weekday'
    END AS day_of_week,
    SUM(price) / SUM(SUM(price)) OVER (PARTITION BY event_type) AS week_ratio
FROM ecom.silver.Dedup_events
WHERE brand IS NOT NULL AND category IS NOT NULL
GROUP BY event_type, 
         CASE WHEN dayofweek(event_date) IN (1,7) THEN 'weekend' ELSE 'weekday' END

In [0]:
CREATE OR REPLACE TABLE ecom.gold.conversion_rate AS
SELECT 
    category,
    CAST(SUM(price) AS bigint) AS total_price,
    CAST(SUM(CASE WHEN event_type = 'purchase' THEN price ELSE 0 END) AS BIGINT) AS total_purchase,
    ROUND((SUM(CASE WHEN event_type = 'purchase' THEN price ELSE 0 END) / NULLIF(SUM(CASE WHEN event_type = 'cart' THEN price ELSE 0 END), 0)) * 100, 2) AS Cart_conversionRate,
    ROUND((SUM(CASE WHEN event_type = 'purchase' THEN price ELSE 0 END) / NULLIF(SUM(CASE WHEN event_type = 'view' THEN price ELSE 0 END), 0)) * 100, 2) AS View_conversionRate
FROM ecom.silver.Dedup_events
WHERE brand IS NOT NULL AND category IS NOT NULL
GROUP BY category

In [0]:
USE CATALOG ecom;
USE SCHEMA gold;
show tables